In [1]:
#Install Cellpose and Dependencies
!pip install "cellpose[all]"


In [2]:
#Import Required Libraries
import os
from cellpose import models, io
from cellpose.gui import gui  # For human-in-the-loop annotation
from IPython.display import display, Image
import matplotlib.pyplot as plt

objc[14410]: Class QT_ROOT_LEVEL_POOL__THESE_OBJECTS_WILL_BE_RELEASED_WHEN_QAPP_GOES_OUT_OF_SCOPE is implemented in both /Users/mattice/miniconda3/lib/libQt5Core.5.15.2.dylib (0x16d7f93b8) and /Users/mattice/miniconda3/lib/python3.11/site-packages/PyQt6/Qt6/lib/QtCore.framework/Versions/A/QtCore (0x340a18620). One of the two will be used. Which one is undefined.
objc[14410]: Class KeyValueObserver is implemented in both /Users/mattice/miniconda3/lib/libQt5Core.5.15.2.dylib (0x16d7f93e0) and /Users/mattice/miniconda3/lib/python3.11/site-packages/PyQt6/Qt6/lib/QtCore.framework/Versions/A/QtCore (0x340a18648). One of the two will be used. Which one is undefined.
objc[14410]: Class RunLoopModeTracker is implemented in both /Users/mattice/miniconda3/lib/libQt5Core.5.15.2.dylib (0x16d7f9430) and /Users/mattice/miniconda3/lib/python3.11/site-packages/PyQt6/Qt6/lib/QtCore.framework/Versions/A/QtCore (0x340a18698). One of the two will be used. Which one is undefined.


In [3]:
# Step 3: Set Up Your Working Directories
print("Step 3: Setting up directories...")
TRAIN_DIR = "./cellpose_training/train"   # Path to training images
TEST_DIR = "./cellpose_training/test"     # Path to test images
OUTPUT_DIR = "./cellpose_training/output" # Output path for models & results

# Create directories if they don't exist
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Directories ready!")


Step 3: Setting up directories...
Directories ready!


In [4]:
# Step 4: Prepare Your Dataset
print("Step 4: Preparing Dataset...")
print("Copy your training images to:", TRAIN_DIR)
print("Copy your test images to:", TEST_DIR)
print("Human-in-the-loop annotation will start soon!")


Step 4: Preparing Dataset...
Copy your training images to: ./cellpose_training/train
Copy your test images to: ./cellpose_training/test
Human-in-the-loop annotation will start soon!


In [1]:
print("Step 5: Launching Cellpose GUI for manual annotations...")
print("Instructions: Annotate images in the training folder and save masks.")

# Run the Cellpose GUI
!python3 -m cellpose

Step 5: Launching Cellpose GUI for manual annotations...
Instructions: Annotate images in the training folder and save masks.
2024-12-19 13:43:04,016 [INFO] WRITING LOG OUTPUT TO /Users/mattice/.cellpose/run.log
2024-12-19 13:43:04,016 [INFO] 
cellpose version: 	3.1.0 
platform:       	darwin 
python version: 	3.12.2 
torch version:  	2.6.0.dev20241126
2024-12-19 13:43:04,571 [INFO] ** TORCH MPS version installed and working. **


Here we can train the model on the annotated images through the GUI

In [ ]:
# Step 6: Train a New Cellpose Model on CPU
print("Step 6: Training the model... (This will take time)")

# Build your model as you want
model = models.CellposeModel(gpu=False, pretrained_model='cyto')  # Force CPU use
train_images = io.imread(os.path.join(TRAIN_DIR, "*.tif"))        # Load images (you can change the way of proceeding extensions)
train_masks = io.imread(os.path.join(TRAIN_DIR, "*.npy"))         # Load masks (you can change the way of proceeding extensions)
model_path = os.path.join(OUTPUT_DIR, "cellpose_custom_model")
print("Training model...")

# Modify your parameter here
model.train(train_images, train_masks, channels=[0,0], save_path=model_path, 
            learning_rate=0.1, n_epochs=7, weight_decay=1e-5)
print(f"Model training complete. Model saved at: {model_path}")

In [ ]:
# Step 7: Run the Model on Test Data
print("Step 7: Running predictions on test data...")
test_images = io.imread(os.path.join(TEST_DIR, "*.tif"))

# Run predictions using the trained model
predictions = model.eval(test_images, channels=[0,0])
io.save_masks(predictions, OUTPUT_DIR, test_images)
print("Predictions complete! Masks saved to:", OUTPUT_DIR)


In [ ]:
# Step 8: Visualize Test Results
print("Step 8: Visualizing predictions...")

for i, image in enumerate(test_images):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title("Original Image")
    
    ax[1].imshow(predictions['masks'][i], cmap='nipy_spectral')
    ax[1].set_title("Predicted Mask")
    
    plt.show()

print("Training and evaluation complete!")
print("Manually check the outputs in:", OUTPUT_DIR)